# Setup

In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.5f}'.format}, suppress = True)

from scipy.interpolate import RegularGridInterpolator

# file names
MEASFILENAME     = "measurements.txt"
MAGFIELDFILENAME = "magfields/bfield_corners.txt"

# constants
v    = 0.002997924580 # speed of light
mass = 0.13957018

# File Reading

In [2]:
# Reads a measurements file and returns an array of measurement locations and measured data.
def read_meas_file(filename):
    Z = []
    m = []
    f = open(filename, "r")
    for line in f:
        meas = line.replace("\n", "").split(' ')
        for i in range(len(meas)):
            meas[i] = float(meas[i])
        Z.append(meas[0])
        m.append([meas[0], meas[1], meas[2], meas[3], meas[4], meas[5], meas[6]])
    f.close()
    Z = np.array(Z)
    m = np.array([np.array(mi) for mi in m])
    return (Z, m)

# Reads a magnetic fields file and returns an array with a [x,y,z] mesh grid and the [bx,by,bz] magfields 
#     measurements.
def unpack_magfield_file(filename):
    B = [[], [], [], [], [], []]
    f = open(filename, "r")
    
    for line in f:
        mag_meas = line.replace("\n", "").split(' ')
        for i in range(len(mag_meas)):
            mag_meas[i] = float(mag_meas[i])
        if mag_meas[0] not in B[0]:
            B[0].append(mag_meas[0])
        if mag_meas[1] not in B[1]:
            B[1].append(mag_meas[1])
        if mag_meas[2] not in B[2]:
            B[2].append(mag_meas[2])
        B[3].append(mag_meas[3])
        B[4].append(mag_meas[4])
        B[5].append(mag_meas[5])

    f.close()
    
    return B

# Packs an array into an array of dest_size sized arrays.
def pack_data(src_arr, dest_size):
    dest_arr = []
    it       = 0
    for d in src_arr:
        if it == 0:
            dest_arr.append([d])
        else:
            dest_arr[-1].append(d)
        it += 1
        if it == dest_size:
            it = 0
    return dest_arr

# Magnetic Field Estimation

In [3]:
# Create interpolators for bx, by and bz based on a set of magnetic field measurements
def create_interpolators(x, y, z, bx, by, bz, bounds_error=True):
    bx_z = pack_data(bx, len(z))
    by_z = pack_data(by, len(z))
    bz_z = pack_data(bz, len(z))

    bx_y = np.array(pack_data(bx_z, len(y)))
    by_y = np.array(pack_data(by_z, len(y)))
    bz_y = np.array(pack_data(bz_z, len(y)))

    bx_interpolator = RegularGridInterpolator((x, y, z), bx_y, method="linear", bounds_error=bounds_error)
    by_interpolator = RegularGridInterpolator((x, y, z), by_y, method="linear", bounds_error=bounds_error)
    bz_interpolator = RegularGridInterpolator((x, y, z), bz_y, method="linear", bounds_error=bounds_error)
    
    return (bx_interpolator, by_interpolator, bz_interpolator)

# Gets the magnetic field (bx,by,bz) at a specific location l (x,y,z)
def get_b(l, B):
    return [B[0]([l[0], l[1], l[2]])[0], B[1]([l[0], l[1], l[2]])[0], B[2]([l[0], l[1], l[2]])[0]]

# Configuration

In [4]:
# prepare the Z planes and measurement arrays
(Z, m) = read_meas_file(MEASFILENAME)

# prepare the interpolators
d = unpack_magfield_file(MAGFIELDFILENAME)
B = create_interpolators(d[0], d[1], d[2], d[3], d[4], d[5], bounds_error=True)

# Kalman Filter
### Runge Kutta 4

In [5]:
# "private" methods called only by Runge Kutta 4
def Ax(tx, ty, bx, by, bz, C):
    return C * ( ty * (tx*bx + bz) - (1 + tx*tx) * by)

def Ay(tx, ty, bx, by, bz, C):
    return C * (-tx * (ty*by + bz) + (1 + ty*ty) * bx)

def delAx_deltx(tx, ty, bx, by, bz, Csq, C, Ax, Ay):
    return tx*Ax/Csq + C * (ty*bx - 2*tx*by)

def delAx_delty(tx, ty, bx, by, bz, Csq, C, Ax, Ay):
    return ty*Ax/Csq + C * (tx*bx + bz)

def delAy_deltx(tx, ty, bx, by, bz, Csq, C, Ax, Ay):
    return tx*Ay/Csq + C * (-ty*by - bz)

def delAy_delty(tx, ty, bx, by, bz, Csq, C, Ax, Ay):
    return ty*Ay/Csq + C * (-tx*by + 2*ty*bx)

def deltx_deltx0_next(qv, tx1, ty1, b0, b1, b2, deltx_deltx0_1, delty_deltx0_1, Csq, C, Ax, Ay):
    return qv * (delAx_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_deltx0_1
            + delAx_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_deltx0_1)

def deltx_delty0_next(qv, tx1, ty1, b0, b1, b2, deltx_delty0_1, delty_delty0_1, Csq, C, Ax, Ay):
    return qv * (delAx_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_delty0_1
            + delAx_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_delty0_1)

def deltx_delq0_next(qv, v, tx1, ty1, b0, b1, b2, deltx_delq0_1, delty_delq0_1, Csq, C, Ax, Ay):
    return v * Ax + qv * (delAx_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_delq0_1
            + delAx_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_delq0_1)

def delty_deltx0_next(qv, tx1, ty1, b0, b1, b2, deltx_deltx0_1, delty_deltx0_1, Csq, C, Ax, Ay):
    return qv * (delAy_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_deltx0_1
            + delAy_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_deltx0_1)

def delty_delty0_next(qv, tx1, ty1, b0, b1, b2, deltx_delty0_1, delty_delty0_1, Csq, C, Ax, Ay):
    return qv * (delAy_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_delty0_1
            + delAy_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_delty0_1)
                 
def delty_delq0_next(qv, v, tx1, ty1, b0, b1, b2, deltx_delq0_1, delty_delq0_1, Csq, C, Ax, Ay):
    return v * Ay + qv * (delAy_deltx(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * deltx_delq0_1
            + delAy_delty(tx1, ty1, b0, b1, b2, Csq, C, Ax, Ay) * delty_delq0_1)

def RK4(k1, k2, k3, k4, h):
    return h/6 * (k1 + 2*k2 + 2*k3 + k4)

In [6]:
# Runs Runge Kutta 4 transport on an initial state vector and covariance matrix si and Si with a step size of h.
# INPUT:
#   si : initial state vector
#   Si : initial covariance matrix
#   h  : step size
#   B  : magnetic field interpolators (bx, by, bz)
# OUTPUT:
#   sf : final state vector
#   Sf : final covariance matrix
def rk4_transport(si, Si, h, B):
    # Unpack the initial state vector
    z0     = si[0]
    x0     = si[1]
    y0     = si[2]
    tx0    = si[3]
    ty0    = si[4]
    q      = si[5]
    d_path = si[7]

    # Jacobian
    u = np.empty([5,5])
    Sf = np.empty([5,5])
    deltx_deltx0_0 = 1.
    deltx_delty0_0 = 0.
    deltx_delq0_0  = 0.
    delty_deltx0_0 = 0.
    delty_delty0_0 = 1.
    delty_delq0_0  = 0.

    # Auxiliary variables
    qv = q*v
    hh = 0.5*h

    # ==- K1 -=============================================================================
    b = get_b([x0, y0, z0], B)

    # Auxiliary variables
    C1sq = 1 + tx0*tx0 + ty0+ty0
    C1   = np.sqrt(C1sq)
    Ax1  = Ax(tx0, ty0, b[0], b[1], b[2], C1)
    Ay1  = Ay(tx0, ty0, b[0], b[1], b[2], C1)

    # State
    x1  = tx0
    y1  = ty0
    tx1 = qv * Ax1
    ty1 = qv * Ay1

    # Jacobian
    delx_deltx0_1 = deltx_deltx0_0
    delx_delty0_1 = deltx_delty0_0
    delx_delq0_1  = deltx_delq0_0

    dely_deltx0_1 = delty_deltx0_0
    dely_delty0_1 = delty_delty0_0
    dely_delq0_1  = delty_delq0_0

    deltx_deltx0_1 = qv * delAx_deltx(tx0, ty0, b[0], b[1], b[2], C1sq, C1, Ax1, Ay1)
    deltx_delty0_1 = qv * delAx_delty(tx0, ty0, b[0], b[1], b[2], C1sq, C1, Ax1, Ay1)
    deltx_delq0_1  = v  * Ax1

    delty_deltx0_1 = qv * delAy_deltx(tx0, ty0, b[0], b[1], b[2], C1sq, C1, Ax1, Ay1)
    delty_delty0_1 = qv * delAy_delty(tx0, ty0, b[0], b[1], b[2], C1sq, C1, Ax1, Ay1)
    delty_delq0_1  = v  * Ay1

    # ==- K2 -=============================================================================
    b = get_b([x0 + hh*x1, y0 + hh*y1, z0 + hh], B)

    # State 1
    x2 = tx0 + hh*tx1
    y2 = ty0 + hh*ty1

    # Auxiliary variables
    C2sq = 1 + x2*x2 + y2*y2
    C2   = np.sqrt(C2sq)
    Ax2  = Ax(x2, y2, b[0], b[1], b[2], C2)
    Ay2  = Ay(x2, y2, b[0], b[1], b[2], C2)

    dtxtx1 = deltx_deltx0_0 + hh*deltx_deltx0_1
    dtxty1 = deltx_delty0_0 + hh*deltx_delty0_1
    dtxtq1 = deltx_delq0_0  + hh*deltx_delq0_1
    dtytx1 = delty_deltx0_0 + hh*delty_deltx0_1
    dtyty1 = delty_delty0_0 + hh*delty_delty0_1
    dtytq1 = delty_delq0_0  + hh*delty_delq0_1

    # State 2
    tx2 = qv * Ax2
    ty2 = qv * Ay2

    # Jacobian
    delx_deltx0_2 = deltx_deltx0_0 + hh*deltx_deltx0_1
    delx_delty0_2 = deltx_delty0_0 + hh*deltx_delty0_1
    delx_delq0_2  = deltx_delq0_0  + hh*deltx_delq0_1

    dely_deltx0_2 = delty_deltx0_0 + hh*delty_deltx0_1
    dely_delty0_2 = delty_delty0_0 + hh*delty_delty0_1
    dely_delq0_2  = delty_delq0_0  + hh*delty_delq0_1

    deltx_deltx0_2 = deltx_deltx0_next(qv, x2, y2, b[0], b[1], b[2], dtxtx1, dtytx1, C2sq, C2, Ax2, Ay2)
    deltx_delty0_2 = deltx_delty0_next(qv, x2, y2, b[0], b[1], b[2], dtxty1, dtyty1, C2sq, C2, Ax2, Ay2)
    deltx_delq0_2  = deltx_delq0_next(qv, v, x2, y2, b[0], b[1], b[2], dtxtq1, dtytq1, C2sq, C2, Ax2, Ay2)

    delty_deltx0_2 = delty_deltx0_next(qv, x2, y2, b[0], b[1], b[2], dtxtx1, dtytx1, C2sq, C2, Ax2, Ay2)
    delty_delty0_2 = delty_delty0_next(qv, x2, y2, b[0], b[1], b[2], dtxty1, dtyty1, C2sq, C2, Ax2, Ay2)
    delty_delq0_2  = delty_delq0_next(qv, v, x2, y2, b[0], b[1], b[2], dtxtq1, dtytq1, C2sq, C2, Ax2, Ay2)

    # ==- K3 -=============================================================================
    get_b([x0 + hh*x2, y0 + hh*y2, z0 + hh], B)

    # State 1
    x3 = tx0 + hh*tx2
    y3 = ty0 + hh*ty2

    # Auxiliary variables
    C3sq = 1 + x3*x3 + y3*y3
    C3   = np.sqrt(C3sq)
    Ax3  = Ax(x3, y3, b[0], b[1], b[2], C3)
    Ay3  = Ay(x3, y3, b[0], b[1], b[2], C3)

    dtxtx2 = deltx_deltx0_0 + hh*deltx_deltx0_2
    dtxty2 = deltx_delty0_0 + hh*deltx_delty0_2
    dtxtq2 = deltx_delq0_0  + hh*deltx_delq0_2
    dtytx2 = delty_deltx0_0 + hh*delty_deltx0_2
    dtyty2 = delty_delty0_0 + hh*delty_delty0_2
    dtytq2 = delty_delq0_0  + hh*delty_delq0_2

    # State 2
    tx3 = qv * Ax3
    ty3 = qv * Ay3

    # Jacobian
    delx_deltx0_3 = deltx_deltx0_0 + hh*deltx_deltx0_2
    delx_delty0_3 = deltx_delty0_0 + hh*deltx_delty0_2
    delx_delq0_3  = deltx_delq0_0  + hh*deltx_delq0_2

    dely_deltx0_3 = delty_deltx0_0 + hh*delty_deltx0_2
    dely_delty0_3 = delty_delty0_0 + hh*delty_delty0_2
    dely_delq0_3  = delty_delq0_0  + hh*delty_delq0_2

    deltx_deltx0_3 = deltx_deltx0_next(qv, x3, y3, b[0], b[1], b[2], dtxtx2, dtytx2, C3sq, C3, Ax3, Ay3)
    deltx_delty0_3 = deltx_delty0_next(qv, x3, y3, b[0], b[1], b[2], dtxty2, dtyty2, C3sq, C3, Ax3, Ay3)
    deltx_delq0_3  = deltx_delq0_next(qv, v, x3, y3, b[0], b[1], b[2], dtxtq2, dtytq2, C3sq, C3, Ax3, Ay3)

    delty_deltx0_3 = delty_deltx0_next(qv, x3, y3, b[0], b[1], b[2], dtxtx2, dtytx2, C3sq, C3, Ax3, Ay3)
    delty_delty0_3 = delty_delty0_next(qv, x3, y3, b[0], b[1], b[2], dtxty2, dtyty2, C3sq, C3, Ax3, Ay3)
    delty_delq0_3  = delty_delq0_next(qv,v, x3, y3, b[0], b[1], b[2], dtxtq2, dtytq2, C3sq, C3, Ax3, Ay3)

    # ==- K4 -=============================================================================
    get_b([x0 + h*x3, y0 + h*y3, z0 + h], B)

    # State 1
    x4 = tx0 + h*tx3
    y4 = ty0 + h*ty3

    # Auxiliary variables
    C4sq = 1 + x4*x4 + y4*y4
    C4   = np.sqrt(C4sq)
    Ax4  = Ax(x4, y4, b[0], b[1], b[2], C4)
    Ay4  = Ay(x4, y4, b[0], b[1], b[2], C4)

    dtxtx3 = deltx_deltx0_0 + hh*deltx_deltx0_3
    dtxty3 = deltx_delty0_0 + hh*deltx_delty0_3
    dtxtq3 = deltx_delq0_0  + hh*deltx_delq0_3
    dtytx3 = delty_deltx0_0 + hh*delty_deltx0_3
    dtyty3 = delty_delty0_0 + hh*delty_delty0_3
    dtytq3 = delty_delq0_0  + hh*delty_delq0_3

    # State 2
    tx4 = qv * Ax4
    ty4 = qv * Ay4

    # Jacobian
    delx_deltx0_4 = deltx_deltx0_0 + h*deltx_deltx0_3
    delx_delty0_4 = deltx_delty0_0 + h*deltx_delty0_3
    delx_delq0_4  = deltx_delq0_0  + h*deltx_delq0_3

    dely_deltx0_4 = delty_deltx0_0 + h*delty_deltx0_3
    dely_delty0_4 = delty_delty0_0 + h*delty_delty0_3
    dely_delq0_4  = delty_delq0_0  + h*delty_delq0_3

    deltx_deltx0_4 = deltx_deltx0_next(qv,  x4, y4, b[0], b[1], b[2], dtxtx3, dtytx3, C4sq, C4, Ax4, Ay4)
    deltx_delty0_4 = deltx_delty0_next(qv,  x4, y4, b[0], b[1], b[2], dtxty3, dtyty3, C4sq, C4, Ax4, Ay4)
    deltx_delq0_4  = deltx_delq0_next(qv,v, x4, y4, b[0], b[1], b[2], dtxtq3, dtytq3, C4sq, C4, Ax4, Ay4)

    delty_deltx0_4 = delty_deltx0_next(qv,  x4, y4, b[0], b[1], b[2], dtxtx3, dtytx3, C4sq, C4, Ax4, Ay4)
    delty_delty0_4 = delty_delty0_next(qv,  x4, y4, b[0], b[1], b[2], dtxty3, dtyty3, C4sq, C4, Ax4, Ay4)
    delty_delq0_4  = delty_delq0_next(qv,v, x4, y4, b[0], b[1], b[2], dtxtq3, dtytq3, C4sq, C4, Ax4, Ay4)

    # ==- RK4 -============================================================================
    # State
    z  = z0  + h
    x  = x0  + RK4(x1,  x2,  x3,  x4,  h)
    y  = y0  + RK4(y1,  y2,  y3,  y4,  h)
    tx = tx0 + RK4(tx1, tx2, tx3, tx4, h)
    ty = ty0 + RK4(ty1, ty2, ty3, ty4, h)

    # Jacobian
    delx_deltx0  = RK4(delx_deltx0_1, delx_deltx0_2, delx_deltx0_3, delx_deltx0_4, h)
    delx_delty0  = RK4(delx_delty0_1, delx_delty0_2, delx_delty0_3, delx_delty0_4, h)
    delx_delq0   = RK4(delx_delq0_1,  delx_delq0_2,  delx_delq0_3,  delx_delq0_4,  h)

    dely_deltx0  = RK4(dely_deltx0_1, dely_deltx0_2, dely_deltx0_3, dely_deltx0_4, h)
    dely_delty0  = RK4(dely_delty0_1, dely_delty0_2, dely_delty0_3, dely_delty0_4, h)
    dely_delq0   = RK4(dely_delq0_1,  dely_delq0_2,  dely_delq0_3,  dely_delq0_4,  h)

    deltx_deltx0 = RK4(deltx_deltx0_1, deltx_deltx0_2, deltx_deltx0_3, deltx_deltx0_4, h) + 1
    deltx_delty0 = RK4(deltx_delty0_1, deltx_delty0_2, deltx_delty0_3, deltx_delty0_4, h)
    deltx_delq0  = RK4(deltx_delq0_1,  deltx_delq0_2,  deltx_delq0_3,  deltx_delq0_4,  h)

    delty_deltx0 = RK4(delty_deltx0_1, delty_deltx0_2, delty_deltx0_3, delty_deltx0_4, h)
    delty_delty0 = RK4(delty_delty0_1, delty_delty0_2, delty_delty0_3, delty_delty0_4, h) + 1
    delty_delq0  = RK4(delty_delq0_1,  delty_delq0_2,  delty_delq0_3,  delty_delq0_4,  h)

    # ==- COVARIANCE MATRIX -==============================================================
    for j1 in range(0,5):
        u[0][j1] = Si[0][j1] + Si[2][j1] * delx_deltx0  + Si[3][j1] * delx_delty0 + Si[4][j1] * delx_delq0
        u[1][j1] = Si[1][j1] + Si[2][j1] * dely_deltx0  + Si[3][j1] * dely_delty0 + Si[4][j1] * dely_delq0
        u[2][j1] = Si[2][j1] + Si[3][j1] * deltx_delty0 + Si[4][j1] * deltx_delq0
        u[3][j1] = Si[2][j1] * delty_deltx0 + Si[3][j1] + Si[4][j1] * delty_delq0
        u[4][j1] = Si[4][j1]

    for i1 in range(0,5):
        Sf[i1][0] = u[i1][0] + u[i1][2] * delx_deltx0 + u[i1][3] * delx_delty0 + u[i1][4] * delx_delq0
        Sf[i1][1] = u[i1][1] + u[i1][2] * dely_deltx0 + u[i1][3] * dely_delty0 + u[i1][4] * dely_delq0
        Sf[i1][2] = u[i1][2] + u[i1][3] * deltx_delty0 + u[i1][4] * deltx_delq0
        Sf[i1][3] = u[i1][2] * delty_deltx0 + u[i1][3] + u[i1][4] * delty_delq0
        Sf[i1][4] = u[i1][4]

    # ==- Q PROCESS NOISE ESTIMATE -=======================================================
    p  = np.abs(1. / q)
    pz = p / np.sqrt(1 + tx * tx + ty * ty)
    px = tx * pz
    py = ty * pz

    # Path length in radiation length units = t/X0 [true path length/ X0]
    # Ar radiation length = 14 cm
    t_ov_X0 = np.sign(h) * h / 14

    # Use particle momentum
    beta = p / np.sqrt(p * p + mass * mass)
    cos_entrenace_angle = np.absolute((x * px + y * py + z * pz) / (np.sqrt(x * x + y * y + z * z) * p))
    path_length = t_ov_X0 / cos_entrenace_angle

    # Highland-Lynch-Dahl formula
    sct_rms = (0.0136 / (beta * p)) * np.sqrt(path_length) * (1 + 0.038 * np.log(path_length))

    cov_txtx = (1 + tx * tx) * (1 + tx * tx + ty * ty) * sct_rms * sct_rms
    cov_tyty = (1 + ty * ty) * (1 + tx * tx + ty * ty) * sct_rms * sct_rms
    cov_txty = tx * ty * (1 + tx * tx + ty * ty) * sct_rms * sct_rms

    if h > 0:
        Sf[2][2] += cov_txtx
        Sf[2][3] += cov_txty
        Sf[3][2] += cov_txty
        Sf[3][3] += cov_tyty

    b_val = np.sqrt(b[0]*b[0] + b[1]*b[1] + b[2]*b[2])
    n_d_path = np.sqrt((x0-x)*(x0-x) + (y0-y)*(y0-y) + h*h) + d_path

    sf = [z, x, y, tx, ty, q, b_val, n_d_path]

    return (sf, Sf)

In [7]:
# Test
#        z           x          y          tx        ty        Q          B         d_path
rk_si = [229.279549, 13.132957, 19.792149, 0.000275, 0.085967, -0.323624, 0.054975, 0.]
rk_Si = [[126.581687, 0.,           0.,       0.,       0.      ],
         [0.,         11458.570644, 0.,       0.,       0.      ],
         [0.,         0.,           0.001116, 0.,       0.      ],
         [0.,         0.,           0.,       0.100980, 0.      ],
         [0.,         0.,           0.,       0.,       0.009548]]

rk4_transport(rk_si, rk_Si, 1., B)

([230.279549,
  13.133226656786823,
  19.87812945068392,
  0.0002644572703414839,
  0.08599356027238665,
  -0.323624,
  0.05029525784313435,
  1.003689548921674],
 array([[ 126.58280,  0.00000,  0.00112,  0.00000,  0.00000],
        [ 0.00000,  11458.67162,  0.00000,  0.10098, -0.00000],
        [ 0.00112,  0.00000,  0.00112,  0.00000,  0.00000],
        [ 0.00000,  0.10098,  0.00000,  0.10098, -0.00000],
        [ 0.00000, -0.00000,  0.00000, -0.00000,  0.00955]]))

### Transport

In [8]:
# transport a state vector s from k-1|k-1 to k|k-1 using Runge Kutta 4
# INPUT:
#   i  : initial state k-1|k-1
#   f  : final state k|k-1
#   si : initial state vector
#   Si : initial covariance matrix
#   Z  : list of measurement planes
# OUTPUT:
#   sf : final state vector
#   Sf : final covariance matrix
def transport(i, f, si, Si, Z):
    sf = si.copy()
    Sf = Si.copy()

    step_size = 1.
    z = Z[i]
    b_at_meas = si[6]

    Z_sign = np.sign(Z[f] - Z[i])

    while Z_sign * sf[0] < Z_sign * Z[f]:
        s = Z_sign * step_size

        if Z_sign * (sf[0]+s) > Z_sign * Z[f]:
            s = Z_sign * np.absolute(Z[f]-sf[0])
            
        sf, Sf = rk4_transport(sf, Sf, s, B)

        if np.absolute(sf[6] - b_at_meas) < 0.0001:
            step_size *= 2

    return (sf, Sf)

In [9]:
# Test
#         z           x          y          tx        ty        Q          B         d_path
tr_si = [229.279549, 13.132957, 19.792149, 0.000275, 0.085967, -0.323624, 0.054975, 0.0]
tr_Si = [[126.581687, 0.000000,     0.000000, 0.000000, 0.000000],
         [0.000000,   11458.570644, 0.000000, 0.000000, 0.000000],
         [0.000000,   0.000000,     0.001116, 0.000000, 0.000000],
         [0.000000,   0.000000,     0.000000, 0.100980, 0.000000],
         [0.000000,   0.000000,     0.000000, 0.000000, 0.009548]]
transport(0, 1, tr_si, tr_Si, Z)

([230.438029,
  13.133268434546947,
  19.891758048175305,
  0.00026279510223036314,
  0.08599777213630581,
  -0.323624,
  0.050300289614831135,
  1.1627544753352694],
 array([[ 126.58318,  0.00000,  0.00129,  0.00000,  0.00000],
        [ 0.00000,  11458.70617,  0.00001,  0.11698, -0.00000],
        [ 0.00129,  0.00001,  0.00112,  0.00000,  0.00000],
        [ 0.00000,  0.11698,  0.00000,  0.10098, -0.00000],
        [ 0.00000, -0.00000,  0.00000, -0.00000,  0.00955]]))

### Filter

In [10]:
# "Private" filter methods
def get_H(y, s, w, l):
    return [1, -np.tan(np.deg2rad(6.*s)) - w * (4./l) * (1 - y/(l/2.))]

def get_h(x, y, s, w, l):
    return x - np.tan(np.radians(6*s)) * y + w*(1 - y/(l/2)) * (1 - y/(l/2))

In [11]:
# filters a state vector and a covariance matrix from k|k-1 to k|k
# INPUT:
#   si   : previous state vector s_{k|k-1}
#   Si   : previous covariance matrix S_{k|k-1}
#   mv   : measurement vector taken at k
#   chi2 : fit's chi^2 error mvured before the filter
# OUTPUT:
#   sf   : next state vector s_{k|k}
#   Sf   : next covariance matrix S_{k|k}
#   chi2 : fit's chi^2 error after the filter
def k_filter(si, Si, mv, chi2):
    if np.absolute(np.linalg.det(Si)) < 1.e-30:
        print("ERROR, INITIAL COVARIANCE MATRIX IS SINGULAR")
        return

    K = [0., 0., 0., 0., 0.]
    V = np.absolute(mv[2])
    H = get_H(si[2], mv[3], mv[6], mv[5])
    
    # Sherman-Morrison formula
    Hv  = np.array([[H[0]], [H[1]], [0], [0], [0]])
    HvT = np.transpose(Hv)

    div = V + np.dot(np.dot(HvT, Si), Hv)[0, 0]
    res = np.dot(np.dot(np.dot(Si, Hv), HvT), Si)
    res = np.true_divide(res, div)
    Sf = np.subtract(Si, res)

    if np.absolute(np.linalg.det(Sf)) < 1.e-30:
        print("ERROR, FINAL COVARIANCE MATRIX IS SINGULAR")
        return

    for j in range(5):
        K[j] = (H[0] * Sf[j][0] + H[1] * Sf[j][1])/V

    # Filter the state vector
    h = get_h(si[1], si[2], mv[3], mv[6], mv[5])
    
    chi2 += (mv[1] - h) * (mv[1] - h) / V

    sf = np.copy(si)
    sf[1] += K[0] * (mv[1] - h)
    sf[2] += K[1] * (mv[1] - h)
    sf[3] += K[2] * (mv[1] - h)
    sf[4] += K[3] * (mv[1] - h)
    sf[5] += K[4] * (mv[1] - h)

    return (sf, Sf, chi2)

In [12]:
# Test
#        z           x          y          tx        ty        Q          B         d_path
fi_si = [233.913469, 12.957073, 21.876995, 0.000158, 0.085952, -0.323624, 0.066656, 1.162750]
fi_Si = [[ 64.491346,  591.978446, 0.003912,  0.012127,  0.000007],
         [591.978446, 5818.911315, 0.012160,  0.352917,  0.000007],
         [  0.003912,    0.012160, 0.001120,  0.000023,  0.000004],
         [  0.012127,    0.352917, 0.000023,  0.100960, -0.000002],
         [  0.000007,    0.000007, 0.000004, -0.000002,  0.009548]]
#        z           x          unc        tilt error     wire_len    wire_max_sag
fi_mv = [233.913469, 10.790751, 13.275960, 1,   0.048943, 114.104084, 0.007913]
fi_chi2  = 0.014741

k_filter(fi_si, fi_Si, fi_mv, fi_chi2)

(array([ 233.91347,  12.97310,  21.72487,  0.00018,  0.08577, -0.32362,
         0.06666,  1.16275]),
 array([[ 64.22388,  594.51787,  0.00359,  0.01521,  0.00001],
        [ 594.51787,  5794.80120,  0.01524,  0.32364,  0.00001],
        [ 0.00359,  0.01524,  0.00112,  0.00003,  0.00000],
        [ 0.01521,  0.32364,  0.00003,  0.10092, -0.00000],
        [ 0.00001,  0.00001,  0.00000, -0.00000,  0.00955]]),
 0.016014661135581646)

### Kalman Fitter

In [13]:
# runs the Kalman filter an I amount of times
# INPUT:
#   Z : list of measurement planes
#   m : list of measurement vectors
#   s : list of state vectors
#   S : list of covariance matrices
#   I : number of iterations to be ran
def run_fitter(Z, m, s, S, I):
    b_chi2 = float("inf")
    K = len(Z)-1

    for i in range(1, I+1):
        c_chi2 = 0
        if i > 1:
            s[0], S[0] = transport(K, 0, s[K], S[K], Z)
        for k in range(0, K):
            s[k+1], S[k+1] = transport(k, k+1, s[k], S[k], Z)
            s[k+1], S[k+1], c_chi2 = k_filter(s[k+1], S[k+1], m[k+1], c_chi2)

        if i <= 1:
            continue
        if (c_chi2 < b_chi2):
            b_chi2 = c_chi2

        print("iteration " + str(i) + " : ", s[K][0:6], " (", str(b_chi2), ")")

# Tests

In [14]:
# config
I = 60 # number of iterations

# state vector and covariance matrix arrays
s = []
S = []

for i in range(len(Z)):
    s.append(np.zeros(8))
    s[i][0] = Z[i]
    S.append(np.zeros((5,5)))

# initial state vector and covariance matrix
#               z      x          y          tx        ty        Q          B         delta_path
s[0] = np.array([Z[0], 13.132957, 19.792149, 0.000275, 0.085967, -0.323624, 0.054975, 0.])
S[0] = np.array(
#     x           y         tx         ty         Q
    [[27.809679,  0.941768, -0.109315, -0.004673,  0.009938],
     [ 0.941768, 85.895073, -0.002793, -0.410291,  0.003917],
     [-0.109315, -0.002793,  0.000481,  0.000020, -0.000130],
     [-0.004673, -0.410291,  0.000020,  0.002811, -0.000046],
     [ 0.009938,  0.003917, -0.000130, -0.000046,  0.000733]]
)

run_fitter(Z, m, s, S, I)

iteration 2 :  [ 528.96033  5.13178  43.19782 -0.04197  0.07361 -0.31656]  ( 20.43052415343432 )
iteration 3 :  [ 528.96033  4.92643  42.13894 -0.04184  0.06821 -0.31644]  ( 20.05403453058865 )
iteration 4 :  [ 528.96033  4.88162  41.34798 -0.04177  0.06430 -0.31640]  ( 19.873561906486152 )
iteration 5 :  [ 528.96033  4.86929  40.73692 -0.04177  0.06136 -0.31639]  ( 19.801083692895794 )
iteration 6 :  [ 528.96033  4.86408  40.25336 -0.04178  0.05909 -0.31641]  ( 19.767344706991793 )
iteration 7 :  [ 528.96033  4.86079  39.86430 -0.04180  0.05731 -0.31643]  ( 19.74835556396834 )
iteration 8 :  [ 528.96033  4.85827  39.54767 -0.04181  0.05589 -0.31646]  ( 19.736649424359854 )
iteration 9 :  [ 528.96033  4.87094  39.28617 -0.04157  0.05474 -0.31650]  ( 19.736649424359854 )
iteration 10 :  [ 528.96033  4.87028  39.07151 -0.04158  0.05382 -0.31655]  ( 19.698185879742265 )
iteration 11 :  [ 528.96033  4.86896  38.89349 -0.04159  0.05308 -0.31659]  ( 19.694071585421785 )
iteration 12 :  [ 528